# Recommendation system model training

### 1) Import relevant libraries

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statistics import median
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


### Faster but less thorough version.

In [22]:
csv_file_path = 'clean_train_reco.csv'
df = pd.read_csv(csv_file_path)

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=UserWarning, message=".*ConvergenceWarning.*")

# Separate features and target variables
prediction_columns = ['fixed_deposits', 'loan', 'credit_card_debit_card', 'account']
X = df.drop(columns=prediction_columns)  # First 86 columns (features)
y = df[prediction_columns]

# Normalize the feature columns
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [23]:
# Create a class for model evaluation
class ModelEvaluator:
    def __init__(self):
        self.models = {}
        
    def train_models(self, X_train, y_train):
        # Define the logistic regression model
        lr_model = LogisticRegression(max_iter=1000, class_weight='balanced', solver='liblinear')

        # Set up hyperparameters for tuning
        param_dist = {
            'C': [0.001, 0.01, 0.1, 1],  # Reduced number of hyperparameters
            'penalty': ['l1', 'l2'],  # Only two regularization types
        }

        # Use StratifiedKFold for cross-validation
        skf = StratifiedKFold(n_splits=3)  # Reduced number of folds

        # Perform random search with cross-validation
        random_search = RandomizedSearchCV(lr_model, param_dist, n_iter=8, cv=skf, scoring='f1_macro', n_jobs=-1)

        for i in range(y_train.shape[1]):
            print(f"Training model for target column {y_train.columns[i]}")
            # Train the model
            random_search.fit(X_train, y_train.iloc[:, i])
            self.models[y_train.columns[i]] = random_search.best_estimator_
            print(f"Best parameters for {y_train.columns[i]}: {random_search.best_params_}")

        return self.models

    def evaluate_model(self, model, X_test, y_test, column_name):
        y_pred = model.predict(X_test)
        print(f"Confusion Matrix for {column_name}:\n", confusion_matrix(y_test[column_name], y_pred))
        print(f"Classification Report for {column_name}:\n", classification_report(y_test[column_name], y_pred))
        
    def evaluate_all_models(self, X_test, y_test):
        for column_name, model in self.models.items():
            self.evaluate_model(model, X_test, y_test, column_name)

In [24]:
y.head

<bound method NDFrame.head of         fixed_deposits  loan  credit_card_debit_card  account
0                    0     0                       0        1
1                    0     0                       0        1
2                    0     1                       0        1
3                    0     0                       0        1
4                    1     0                       0        1
...                ...   ...                     ...      ...
119995               0     0                       0        0
119996               0     0                       0        0
119997               0     0                       0        1
119998               0     0                       0        1
119999               1     0                       1        1

[120000 rows x 4 columns]>

In [27]:
# Create and train the models
model_evaluator = ModelEvaluator()
models = model_evaluator.train_models(X_train, y_train)

# Evaluate all trained models
model_evaluator.evaluate_all_models(X_test, y_test)

# After training, you can use the models to make predictions on new data
def predict_proba(new_data):
    new_data_normalized = scaler.transform(new_data)
    probabilities = {}
    for column_name, model in model_evaluator.models.items():
        probabilities[column_name] = model.predict_proba(new_data_normalized)[:, 1]  # Probability of class 1
    return probabilities


Training model for target column fixed_deposits


/Users/carlychinsekyi/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for fixed_deposits: {'penalty': 'l1', 'C': 1}
Training model for target column loan


/Users/carlychinsekyi/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for loan: {'penalty': 'l2', 'C': 1}
Training model for target column credit_card_debit_card


/Users/carlychinsekyi/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for credit_card_debit_card: {'penalty': 'l2', 'C': 1}
Training model for target column account


/Users/carlychinsekyi/miniconda3/envs/dev/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for account: {'penalty': 'l1', 'C': 1}
Confusion Matrix for fixed_deposits:
 [[15069  5487]
 [  315  3129]]
Classification Report for fixed_deposits:
               precision    recall  f1-score   support

           0       0.98      0.73      0.84     20556
           1       0.36      0.91      0.52      3444

    accuracy                           0.76     24000
   macro avg       0.67      0.82      0.68     24000
weighted avg       0.89      0.76      0.79     24000

Confusion Matrix for loan:
 [[13817  7132]
 [  114  2937]]
Classification Report for loan:
               precision    recall  f1-score   support

           0       0.99      0.66      0.79     20949
           1       0.29      0.96      0.45      3051

    accuracy                           0.70     24000
   macro avg       0.64      0.81      0.62     24000
weighted avg       0.90      0.70      0.75     24000

Confusion Matrix for credit_card_debit_card:
 [[10375  7812]
 [  580  5233]]
Classifica

In [28]:
models

{'fixed_deposits': LogisticRegression(C=1, class_weight='balanced', max_iter=1000, penalty='l1',
                    solver='liblinear'),
 'loan': LogisticRegression(C=1, class_weight='balanced', max_iter=1000,
                    solver='liblinear'),
 'credit_card_debit_card': LogisticRegression(C=1, class_weight='balanced', max_iter=1000,
                    solver='liblinear'),
 'account': LogisticRegression(C=1, class_weight='balanced', max_iter=1000, penalty='l1',
                    solver='liblinear')}

### This model takes a while to run, its the more thorough version. Select the code and command + / for mac to undo the # . 

In [26]:
# # Separate features and target variables
# prediction_columns = ['fixed_deposits', 'loan', 'credit_card_debit_card', 'account']
# X = df.drop(columns=prediction_columns)  # First 86 columns (features)
# y = df[prediction_columns]

# # Normalize the feature columns
# scaler = StandardScaler()
# X_normalized = scaler.fit_transform(X)

# # Should use, stratified train-test split to maintain class distribution, but errors encountered ()
# # one of the classes is has too few rows
# X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# # Create a class for model evaluation
# class ModelEvaluator:
#     def __init__(self):
#         self.models = {}
        
#     def train_models(self, X_train, y_train):
#         # Define the logistic regression model
#         lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')

#         # Set up hyperparameters for tuning
#         param_grid = {
#             'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
#             'penalty': ['l1', 'l2', 'elasticnet', 'none'],  # Regularization type
#             'solver': ['liblinear', 'saga']  # Solver to use
#         }

#         # Use StratifiedKFold for cross-validation
#         skf = StratifiedKFold(n_splits=5)

#         # Perform grid search with cross-validation
#         grid_search = GridSearchCV(lr_model, param_grid, cv=skf, scoring='f1_macro', n_jobs=-1)
        
#         for i in range(y_train.shape[1]):
#             print(f"Training model for target column {y_train.columns[i]}")
#             # Train the model
#             grid_search.fit(X_train, y_train.iloc[:, i])
#             self.models[y_train.columns[i]] = grid_search.best_estimator_
#             print(f"Best parameters for {y_train.columns[i]}: {grid_search.best_params_}")



#     def evaluate_model(self, model, X_test, y_test, column_name):
#         y_pred = model.predict(X_test)
#         print(f"Confusion Matrix for {column_name}:\n", confusion_matrix(y_test[column_name], y_pred))
#         print(f"Classification Report for {column_name}:\n", classification_report(y_test[column_name], y_pred))
        
#     def evaluate_all_models(self, X_test, y_test):
#         for column_name, model in self.models.items():
#             self.evaluate_model(model, X_test, y_test, column_name)

# # Create and train the models
# model_evaluator = ModelEvaluator()
# model_evaluator.train_models(X_train, y_train)

# # Evaluate all trained models
# model_evaluator.evaluate_all_models(X_test, y_test)

# # After training, you can use the models to make predictions on new data
# def predict_proba(new_data):
#     new_data_normalized = scaler.transform(new_data)
#     probabilities = {}
#     for column_name, model in model_evaluator.models.items():
#         probabilities[column_name] = model.predict_proba(new_data_normalized)[:, 1]  # Probability of class 1
#     return probabilities


